# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [5]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [6]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [7]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [158]:
#s_files = get_files("data/song_data")
#s_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json'

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, typ = 'series')
df.head(20)

num_songs                               1
artist_id              ARD0S291187B9B7BF5
artist_latitude                      None
artist_longitude                     None
artist_location                      Ohio
artist_name                       Rated R
song_id                SOMJBYD12A6D4F8557
title               Keepin It Real (Skit)
duration                          114.782
year                                    0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df.filter(['song_id', 'title', 'artist_id', 'year', 'duration']).values.tolist()
song_data

['SOMJBYD12A6D4F8557',
 'Keepin It Real (Skit)',
 'ARD0S291187B9B7BF5',
 0,
 114.78159]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df.filter(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude' ]).values.tolist()
artist_data

['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files("data/log_data")

In [ ]:
filepath = log_files[0]
filepath

In [6]:
df = pd.read_json(filepath, lines = True)
#df = pd.read_json(filepath, lines = True, typ = 'series')
#df.head(50)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [7]:
df = df[df["page"]=="NextSong"]
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
5,California Swag District,Logged In,Lily,F,43,Koch,239.17669,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Teach Me How To Dougie,200,1542846506796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
6,Miike Snow,Logged In,Lily,F,44,Koch,385.35791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Silvia,200,1542846745796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
7,Katy Perry,Logged In,Lily,F,45,Koch,179.40853,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,I Kissed A Girl,200,1542847130796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
8,Sikth,Logged In,Lily,F,46,Koch,250.53995,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Peep Show,200,1542847309796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
9,Lily Allen,Logged In,Lily,F,47,Koch,199.88853,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Not Fair,200,1542847559796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [52]:
t = pd.to_datetime(df['ts'], unit='ms')
#t = df.filter(['ts'])
t.head()

0   2018-11-22 00:03:52.796
1   2018-11-22 00:09:10.796
2   2018-11-22 00:12:06.796
3   2018-11-22 00:15:41.796
4   2018-11-22 00:23:40.796
Name: ts, dtype: datetime64[ns]

In [128]:
#time_data = [t.values.tolist(), t.tolist(), t.dt.hour.values.tolist(),t.dt.day.values.tolist(),t.dt.week.values.tolist(), t.dt.month.values.tolist(), t.dt.year.values.tolist(), t.dt.weekday.values.tolist()]
#column_labels = ["ts","ts_date_time","hour","day","week","month","year","weekday"]
time_data =  [t.tolist(), t.dt.hour.values.tolist(),t.dt.day.values.tolist(),t.dt.week.values.tolist(), t.dt.month.values.tolist(), t.dt.year.values.tolist(), t.dt.weekday.values.tolist()]
column_labels = ["ts_date_time","hour","day","week","month","year","weekday"]
time_data
#column_labels

[[Timestamp('2018-11-22 00:03:52.796000'),
  Timestamp('2018-11-22 00:09:10.796000'),
  Timestamp('2018-11-22 00:12:06.796000'),
  Timestamp('2018-11-22 00:15:41.796000'),
  Timestamp('2018-11-22 00:23:40.796000'),
  Timestamp('2018-11-22 00:28:26.796000'),
  Timestamp('2018-11-22 00:32:25.796000'),
  Timestamp('2018-11-22 00:38:50.796000'),
  Timestamp('2018-11-22 00:41:49.796000'),
  Timestamp('2018-11-22 00:45:59.796000'),
  Timestamp('2018-11-22 00:49:18.796000'),
  Timestamp('2018-11-22 00:57:33.796000'),
  Timestamp('2018-11-22 01:01:46.796000'),
  Timestamp('2018-11-22 01:05:35.796000'),
  Timestamp('2018-11-22 01:12:11.796000'),
  Timestamp('2018-11-22 01:17:37.796000'),
  Timestamp('2018-11-22 01:21:37.796000'),
  Timestamp('2018-11-22 01:24:59.796000'),
  Timestamp('2018-11-22 01:30:05.796000'),
  Timestamp('2018-11-22 01:34:51.796000'),
  Timestamp('2018-11-22 01:38:07.796000'),
  Timestamp('2018-11-22 01:40:18.796000'),
  Timestamp('2018-11-22 01:44:29.796000'),
  Timestamp

In [129]:
#time_df = pd.DataFrame(time_data).transpose()
#time_df.columns = column_labels
#ts_dict = dict(zip(column_labels, time_data))

time_df =pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head(10)
#ts_dict
#type(time_data)

,ts_date_time,hour,day,week,month,year,weekday
0,2018-11-22 00:03:52.796,0,22,47,11,2018,3
1,2018-11-22 00:09:10.796,0,22,47,11,2018,3
2,2018-11-22 00:12:06.796,0,22,47,11,2018,3
3,2018-11-22 00:15:41.796,0,22,47,11,2018,3
4,2018-11-22 00:23:40.796,0,22,47,11,2018,3
5,2018-11-22 00:28:26.796,0,22,47,11,2018,3
6,2018-11-22 00:32:25.796,0,22,47,11,2018,3
7,2018-11-22 00:38:50.796,0,22,47,11,2018,3
8,2018-11-22 00:41:49.796,0,22,47,11,2018,3
9,2018-11-22 00:45:59.796,0,22,47,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [130]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [133]:
user_df = df.filter(["userId", "firstName", "lastName", "gender", "level"]).drop_duplicates()
#user_df.head(50)

,userId,firstName,lastName,gender,level
0,15,Lily,Koch,F,paid
39,26,Ryan,Smith,M,free
42,50,Ava,Robinson,F,free
43,97,Kate,Harrell,F,paid
72,101,Jayden,Fox,M,free
74,63,Ayla,Johnson,F,free
75,32,Lily,Burns,F,free
76,23,Morris,Gilmore,M,free
87,83,Stefany,White,F,free
89,76,Jayden,Duffy,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [134]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [8]:
#df = df.head(0)
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    #cur.execute("SELECT songs.song_id, artists.artist_id FROM songs JOIN artists ON songs.artist_id = artists.artist_id")
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.itemInSession, pd.to_datetime(row.ts, unit='ms'),  row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    songplay_data
    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

songplay_data

[2,
 Timestamp('2018-11-22 22:11:34.796000'),
 '80',
 'paid',
 None,
 None,
 823,
 'Portland-South Portland, ME',
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [9]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.